# Preparing the data

- Downloading the Features of 200 manually coded companies from the S&P1500
- The tickers are already coded into one of the four classes 
    - 1 = Orchestrator Core
    - 2 = Orchestrator Periph.
    - 3 = Complementor Core
    - 4 = Complementor Peripheral
    
- We import the necessary libraries and read the Excel file containing the company list with their corresponding tickers and labels.

In [43]:
base_url = "https://financialmodelingprep.com/api"
API_KEY = "ieZWryBMhiEhowJQXvvJBSo8rcJfvMVi"

In [44]:
import pandas as pd
tickers_df = pd.read_excel("Transitions_Cases_Literatur.xlsx")
tickers_df.head()

,Ticker
0,HP
1,IBM
2,CSCO
3,PYPL
4,INTC


## Importing the Balance Sheet Data, Income Statement and CashFlowStatement Data

Next, we import the Balance Sheet, Income Statement, and Cash Flow Statement data for each company using their tickers.

# Function for Importing Data using the V3 Version of the API

In [45]:
import pandas as pd
import requests
from tqdm import tqdm

def fetch_and_concat_data(tickers, statement_type, base_url, version, API_KEY):
    dfs = []  # List to store DataFrames for each company

    # Initialize tqdm progress bar
    progress_bar = tqdm(tickers, desc=f"Fetching {statement_type} data", unit=" ticker")
    
    for ticker in progress_bar:
        progress_bar.set_postfix({"Ticker": ticker})
        url = f"{base_url}/{version}/{statement_type}/{ticker}?apikey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        df = pd.DataFrame(data)
        df['Ticker'] = ticker  # Adding a column for ticker symbol
        dfs.append(df)
        
            
    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(dfs, ignore_index=True)
    final_df.head()
    return final_df

# Assuming tickers_df is your DataFrame containing tickers and labels
balancesheet_df = fetch_and_concat_data(tickers_df["Ticker"], "balance-sheet-statement", base_url, 'v3', API_KEY)
incomestatement_df = fetch_and_concat_data(tickers_df["Ticker"], "income-statement", base_url,'v3', API_KEY)
cashflowstatement_df = fetch_and_concat_data(tickers_df["Ticker"], "cash-flow-statement", base_url,'v3', API_KEY)
keymetrics_df = fetch_and_concat_data(tickers_df["Ticker"], 'key-metrics', base_url,'v3', API_KEY)

Fetching key-metrics data: 100%|██████████| 5/5 [00:04<00:00,  1.23 ticker/s, Ticker=INTC]


In [46]:
# Check which symbols are missing in balancesheet_df.symbol
# missing_symbols = tickers_df[~tickers_df['Ticker'].isin(employeecount_df['symbol'])]

# Print the missing symbols
# print("Symbols missing in balancesheet_df:")
# print(missing_symbols)

In [47]:
import pandas as pd

# Assuming dfs is a list of DataFrames: balancesheet_df, incomestatement_df, cashflowstatement_df, keymetrics_df
dfs = [balancesheet_df, incomestatement_df, cashflowstatement_df, keymetrics_df]

# Process each DataFrame in the list
for df in dfs:
    # Check if 'date' column exists to extract 'calendarYear'
    if 'calendarYear' in df.columns:
        df['calendarYear'] = pd.DatetimeIndex(df['calendarYear']).year  # Extract year from date
        # Filter rows where 'calendarYear' is between 2007 and 2023
        df = df[df['calendarYear'].between(1980, 2023)]


In [48]:
balancesheet_df = balancesheet_df[balancesheet_df['calendarYear'].between(1980, 2023)]
incomestatement_df = incomestatement_df[incomestatement_df['calendarYear'].between(1980, 2023)]
cashflowstatement_df = cashflowstatement_df[cashflowstatement_df['calendarYear'].between(1980, 2023)]
keymetrics_df = keymetrics_df[keymetrics_df['calendarYear'].between(1980, 2023)]

In [49]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import requests

def fetch_and_concat_data_v4(tickers,statement_type, base_url, version, API_KEY):
    dfs = []  # List to store DataFrames for each company
    years = range(2023, 1999, -1)  # Define range of years for synthetic data

    # Initialize tqdm progress bar
    progress_bar = tqdm(tickers, desc=f"Fetching {statement_type} data", unit="ticker")
    
    for ticker in progress_bar:
        progress_bar.set_postfix({"Ticker": ticker})
        url = f"{base_url}/{version}/{statement_type}?symbol={ticker}&apikey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        df = pd.DataFrame(data)
        df['Ticker'] = ticker  # Adding a column for ticker symbol
        dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(dfs, ignore_index=True)
    return final_df

employeecount_df = fetch_and_concat_data_v4(tickers_df["Ticker"],'historical/employee_count', base_url, 'v4', API_KEY)


Fetching historical/employee_count data: 100%|██████████| 5/5 [00:04<00:00,  1.22ticker/s, Ticker=INTC]


In [50]:
import pandas as pd
import numpy as np

def fill_missing_values(employeecount_df, tickers_df):
    # Loop through each ticker in tickers_df
    for index, row in tickers_df.iterrows():
        symbol = row['Ticker']

        # Check if the symbol is in employeecount_df
        if symbol not in employeecount_df['symbol'].values:
            years = range(2023, 1980, -1)  # Define range of years for synthetic data
            synthetic_data = {
                'symbol': [symbol] * len(years),
                'cik': [np.nan] * len(years),
                'acceptanceTime': [np.nan] * len(years),
                'periodOfReport': [f"{year}" for year in years],  # Example period of report dates
                'formType': [np.nan] * len(years),
                'filingDate': [np.nan] * len(years),
                'employeeCount': [np.nan] * len(years),
                'source': [np.nan] * len(years),
            }
            df = pd.DataFrame(synthetic_data)
            employeecount_df = pd.concat([employeecount_df, df], ignore_index=True)
        
    return employeecount_df


employeecount_df = fill_missing_values(employeecount_df, tickers_df)

In [51]:
employeecount_df.rename(columns={"periodOfReport": 'calendarYear'}, inplace = True)

In [52]:
employeecount_df['calendarYear'] = pd.to_datetime(employeecount_df['calendarYear']).dt.year

In [53]:
employeecount_df = employeecount_df[(employeecount_df['calendarYear'] >= 1980) & (employeecount_df['calendarYear'] <= 2023)]

In [54]:
employeecount_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 0 to 127
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   symbol          128 non-null    object
 1   cik             128 non-null    object
 2   acceptanceTime  128 non-null    object
 3   calendarYear    128 non-null    int64 
 4   companyName     128 non-null    object
 5   formType        128 non-null    object
 6   filingDate      128 non-null    object
 7   employeeCount   128 non-null    int64 
 8   source          128 non-null    object
 9   Ticker          128 non-null    object
dtypes: int64(2), object(8)
memory usage: 11.0+ KB


# Import Segement Data
The segement data has the data type dictionaries. Therefore we need to request the data differenetly

In [55]:
import pandas as pd
import json
from urllib.request import urlopen
import certifi



def get_jsonparsed_data(url):
    with urlopen(url, cafile=certifi.where()) as response:
        data = response.read().decode("utf-8")
    return json.loads(data)

# Base URL and API key configuration
base_url = "https://financialmodelingprep.com/api/v4"
api_key = "ieZWryBMhiEhowJQXvvJBSo8rcJfvMVi"  # Replace 'YOUR_API_KEY' with your actual API key


def fetch_ticker_data(tickers_df, base_url, datatype, api_key, endpoint_params=""):
    
    
    results = []
    for index, row in tickers_df.iterrows():
        symbol = row['Ticker']
        url = f"{base_url}/{datatype}?symbol={symbol}&structure=flat{endpoint_params}&apikey={api_key}"
        try:
            data = get_jsonparsed_data(url)
            results.append({'Ticker': symbol, 'Data': data})
        except Exception as e:
            print(f"Failed to fetch data for {symbol}: {e}")

    return pd.DataFrame(results)


geo_revenue_df = fetch_ticker_data(tickers_df, base_url, "revenue-geographic-segmentation",api_key)
product_revenue_df = fetch_ticker_data(tickers_df, base_url,'revenue-product-segmentation',api_key)

C:\Users\morit\AppData\Local\Temp\ipykernel_33924\1466956216.py:9: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  with urlopen(url, cafile=certifi.where()) as response:
C:\Users\morit\AppData\Local\Temp\ipykernel_33924\1466956216.py:9: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  with urlopen(url, cafile=certifi.where()) as response:


In [56]:
geo_revenue_df

,Ticker,Data
0,HP,"[{'2023-09-30': {'ARGENTINA': 137420000, 'AUST..."
1,IBM,"[{'2023-12-31': {'Americas': 31666000000, 'Asi..."
2,CSCO,"[{'2023-07-29': {'UNITED STATES': 33447000000,..."
3,PYPL,[{'2023-12-31': {'Countries Other Than US And ...
4,INTC,[{'2023-12-30': {'China (Including Hong Kong)'...


In [57]:
import pandas as pd

def aggregate_category_counts(df):
    # List to hold the aggregated data
    results = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        Ticker = row['Ticker']
        data_entries = row['Data']

        if not data_entries:  # Check if the data_entries list is empty
            # Generate rows for each year from 2023 to 2007
            for year in range(2023, 1980, -1):
                results.append({
                    'Ticker': Ticker,
                    'calendarYear': year,
                    'Number of Product Segments': None
                })
        else:
            # Process each year's data in the list
            for entry in data_entries:
                for year_date, revenues in entry.items():
                    # Extract the year part from the date string
                    year = pd.to_datetime(year_date).year

                    # Count the number of unique product categories for the current year
                    num_categories = len(revenues)

                    # Append the result as a new row in the results list
                    results.append({
                        'Ticker': Ticker,
                        'calendarYear': year,
                        'Number of Product Segments': num_categories
                    })

    # Convert the results list to a DataFrame
    return pd.DataFrame(results)

# Example usage assuming product_revenue_df and geo_revenue_df are defined
product_revenue_df = aggregate_category_counts(product_revenue_df)
geo_revenue_df = aggregate_category_counts(geo_revenue_df)


In [58]:
# Renaming 'Ticker' column to 'Symbol' in the DataFrame
product_revenue_df = product_revenue_df.rename(columns={'Ticker': 'symbol'})
geo_revenue_df = geo_revenue_df.rename(columns={'Ticker': 'symbol'})

In [59]:
product_revenue_df = product_revenue_df[product_revenue_df['calendarYear'].between(1980, 2023)]
geo_revenue_df = geo_revenue_df[geo_revenue_df['calendarYear'].between(1980,2023)]

# Preparing the Data, Filtering Out only the necessary collumns

After fetching the data, we filter out only the necessary columns from each DataFrame

In [60]:
balancesheet_df = balancesheet_df[['calendarYear', 'symbol','totalCurrentAssets','totalNonCurrentAssets', 
                                   'totalAssets', 'totalCurrentLiabilities','totalNonCurrentLiabilities',
                                   'totalLiabilities']]


incomestatement_df = incomestatement_df[['calendarYear', 'symbol','revenue', 'costOfRevenue', 'grossProfit',
                                       'operatingExpenses', 'ebitda']]

cashflowstatement_df = cashflowstatement_df[['calendarYear', 'symbol', 'acquisitionsNet','investmentsInPropertyPlantAndEquipment',   
                                            'commonStockIssued','debtRepayment']]

keymetrics_df = keymetrics_df[['calendarYear', 'symbol' ,"debtToEquity", 'debtToAssets', 'marketCap',                              
                               'workingCapital', 'daysOfInventoryOnHand']]

employeecount_df = employeecount_df[['calendarYear', 'symbol', 'employeeCount']]

## Balance Sheet Ratios 

- (Assets/Total Assets) * 100
- (Liabilities/Total Liabilities) * 100 

In [61]:
def calculate_feature_ratios(df, revenue_column='grossProfit'):
    
    # Create a copy of the original DataFrame to avoid modifying the original data
    df_with_ratios = df.copy()
    
    # Calculate ratios for assets
    asset_columns = ['totalCurrentAssets','totalNonCurrentAssets']
    
    total_assets = df_with_ratios[asset_columns].sum(axis=1)
    
    for feature_column in asset_columns:
        df_with_ratios[f'{feature_column}_to_totalAssets_ratio'] = (df_with_ratios[feature_column] / total_assets) * 100
    
    # Calculate ratios for liabilities
    liability_columns = ['totalCurrentLiabilities','totalNonCurrentLiabilities']
    
    total_liabilities = df_with_ratios[liability_columns].sum(axis=1)
    
    for feature_column in liability_columns:
        df_with_ratios[f'{feature_column}_to_totalLiabilities_ratio'] = (df_with_ratios[feature_column] / total_liabilities) * 100
    
    # Drop the original features from the DataFrame
    df_with_ratios.drop(columns=asset_columns + liability_columns, inplace=True)
    
    return df_with_ratios

# Usage example:
balancesheet_ratios_df = calculate_feature_ratios(balancesheet_df)                     

# Cashflow / Income Statement Ratios

- (Columns / Revenue) * 100 

Used Columns
- *incomestatement_columns* = ['costOfRevenue', 'grossProfit', 'researchAndDevelopmentExpenses', 
                               'sellingGeneralAndAdministrativeExpenses', 'operatingExpenses', 
                               'costAndExpenses', 'ebitda', 'operatingIncome']
                               
- *cashflow_columns* = ['netCashProvidedByOperatingActivities', 'netCashUsedForInvestingActivites',
                        'investmentsInPropertyPlantAndEquipment', 'freeCashFlow']

In [62]:

def calculate_feature_ratios(df, revenue_column='grossProfit', revenue_df=None):
    # Verify input DataFrame and revenue DataFrame are provided
    if df is None or revenue_df is None:
        raise ValueError("Both 'df' and 'revenue_df' must be provided.")
    
    # Verify the revenue column exists in revenue_df
    if revenue_column not in revenue_df.columns:
        raise ValueError(f"The revenue column '{revenue_column}' was not found in the revenue DataFrame.")
    
    # Create a copy of the original DataFrame to avoid modifying the original data
    df_with_ratios = df.copy()
    
    # Define the columns for which to calculate ratios related to income statements
    incomestatement_columns = ['revenue', 'costOfRevenue', 'grossProfit', 'operatingExpenses', 'ebitda',]
    
    # Calculate ratios for income statement features
    for feature_column in incomestatement_columns:
        if feature_column in df_with_ratios.columns:
            df_with_ratios[f'{feature_column}_to_Revenue_ratio'] = (df_with_ratios[feature_column] / revenue_df[revenue_column]) * 100
    
    # Define the columns for cash flow calculation
    cashflow_columns = ['acquisitionsNet','investmentsInPropertyPlantAndEquipment','commonStockIssued','debtRepayment',]
    
    # Calculate ratios for cash flow features
    for feature_column in cashflow_columns:
        if feature_column in df_with_ratios.columns:
            df_with_ratios[f'{feature_column}_to_Revenue_ratio'] = (df_with_ratios[feature_column] / revenue_df[revenue_column]) * 100
    
    return df_with_ratios

# Corrected example usage:
cashflowstatement_ratios_df = calculate_feature_ratios(cashflowstatement_df, revenue_column='revenue', revenue_df=incomestatement_df)
incomestatement_ratios_df = calculate_feature_ratios(incomestatement_df, revenue_column='revenue', revenue_df=incomestatement_df)

In [63]:
cashflowstatement_ratios_df = cashflowstatement_ratios_df.reset_index()
incomestatement_ratios_df = incomestatement_ratios_df.reset_index()

# Key Metrics Ratio 

- Here we just need the Ratio working capital / Revenue 

In [64]:
keymetrics_df
keymetrics_df["workingCapital_to_revenue_Ratio"] = (keymetrics_df["workingCapital"]/incomestatement_df["revenue"])*100
keymetrics_ratios_df = keymetrics_df.copy()
keymetrics_ratios_df.drop("workingCapital", axis = 1, inplace=True)

# Employee Count / Revenue Measure

In [65]:
incomestatement_df_red = incomestatement_df[["calendarYear", "symbol", "revenue"]]
employeecount_df[["calendarYear", "symbol"]] = employeecount_df[["calendarYear", "symbol"]].astype("object")

In [66]:
import pandas as pd

# Assuming employeecount_df and incomestatement_df are your DataFrames

# Convert calendarYear and symbol columns to string and trim any whitespaces
for df in [employeecount_df, incomestatement_df]:
    df['calendarYear'] = df['calendarYear'].astype(str).str.strip()
    df['symbol'] = df['symbol'].astype(str).str.strip()

# Filter the incomestatement_df to include only the necessary columns
incomestatement_reduced = incomestatement_df[['calendarYear', 'symbol', 'revenue']]

# Perform the merge using an inner join
employeecount_ratios_df = pd.merge(employeecount_df, incomestatement_reduced, on=['calendarYear', 'symbol'], how='inner')
employeecount_ratios_df["employee_revenue_ratio"] = (employeecount_ratios_df["employeeCount"]/employeecount_ratios_df["revenue"])*100
employeecount_ratios_df.head()

,calendarYear,symbol,employeeCount,revenue,employee_revenue_ratio
0,2023,HP,6200,2872421000,0.000216
1,2022,HP,7000,2058944000,0.000340
2,2021,HP,5444,1218568000,0.000447
3,2020,HP,3634,1773927000,0.000205
4,2019,HP,7767,2798490000,0.000278


## Analyst Recommendations

\begin{align*}
\text{WARS} = \frac{(3 \times \text{analystRatingsStrongBuy}) + (2 \times \text{analystRatingsBuy}) + (\text{analystRatingsHold}) - (2 \times \text{analystRatingsSell}) - (3 \times \text{analystRatingsStrongSell})}{\text{Total Ratings}}
\end{align*}


Assign Weights to Each Category:
Strong Buy: +3
Buy: +2
Hold: +1
Sell: -2
Strong Sell: -3


- With this Ratio, we can evaluate the sentiment of analysts, 


## Partners

In [67]:
balancesheet_df.head()

,calendarYear,symbol,totalCurrentAssets,totalNonCurrentAssets,totalAssets,totalCurrentLiabilities,totalNonCurrentLiabilities,totalLiabilities
0,2023,HP,1006625000,3375331000,4381956000,418931000,1191082000,1610013000
1,2022,HP,1002944000,3352587000,4355531000,394810000,1195249000,1590059000
2,2021,HP,1586566000,3447562000,5034128000,866306000,1255204000,2121510000
3,2020,HP,963327000,3866294000,4829621000,219136000,1291971000,1511107000
4,2019,HP,1115086000,4724429000,5839515000,410238000,1417054000,1827292000


# Dropping Columns

Cashflow-Statements Collumn Drops

In [68]:
cs_2_drop = cashflowstatement_ratios_df[['acquisitionsNet','investmentsInPropertyPlantAndEquipment','commonStockIssued',
                                        'debtRepayment']]
cashflowstatement_ratios_df.drop(columns=cs_2_drop, axis=1, inplace=True )

In [69]:
is_2_drop = incomestatement_ratios_df[["revenue", 'costOfRevenue', 'grossProfit', 'operatingExpenses', 'ebitda' ]]
incomestatement_ratios_df.drop(columns=is_2_drop, axis = 1, inplace=True)

In [70]:
bs_2_drop = balancesheet_df[["totalAssets", "totalLiabilities"]]
balancesheet_ratios_df.drop(columns=bs_2_drop, axis=1, inplace=True)

In [71]:
ec_2_drop = employeecount_ratios_df[["revenue", "employeeCount"]]
employeecount_ratios_df.drop(columns=ec_2_drop, axis =1, inplace = True)

# Final Financial Ratios

- cashflowstatement_ratios_df
- incomestatement_ratios_df
- balancesheet_ratios_df
- keymetrics_ratios_df
- employeecount_ratios_df
- product_revenue_df
- geo_revenue_df

In [72]:
dfs = [cashflowstatement_ratios_df, incomestatement_ratios_df, balancesheet_ratios_df, keymetrics_ratios_df, 
          employeecount_ratios_df, product_revenue_df, geo_revenue_df]

## Imputing missing values using Regression
- When we inspect the distinct dataframes, we can observe that they have different lengths, indicating that there are missing values.

- However, to merge the dfs we need the same lenghts an no missing values, thus we use regression as an imputation method

- The following function checks for every ticker if there is a missing value (year) creates and fills the row with the value obtained from the regression

In [73]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def fill_missing_years(df, start_year=1980, end_year=2023):
    # Ensure 'calendarYear' is treated as integers
    df['calendarYear'] = pd.to_numeric(df['calendarYear'], errors='coerce').fillna(0).astype(int)
    
    # Create a list of all years in the range
    all_years = list(range(start_year, end_year + 1))
    
    # Get all unique tickers
    all_tickers = df['symbol'].unique()
    
    # Create a new dataframe with all combinations of tickers and years
    full_index = pd.MultiIndex.from_product([all_tickers, all_years], names=['symbol', 'calendarYear'])
    full_df = pd.DataFrame(index=full_index).reset_index()
    
    # Merge the original dataframe with the full dataframe
    df_filled = full_df.merge(df, on=['symbol', 'calendarYear'], how='left')
    
    return df_filled

def impute_missing_data(df):
    # Ensure DataFrame has unique symbol-year combinations
    df = df.drop_duplicates(subset=['symbol', 'calendarYear'], keep='first')
    
    # Fill missing years
    df = fill_missing_years(df)
    
    # Replace infinite values with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Group by ticker symbol and impute missing data for numeric columns using regression
    for symbol in df['symbol'].unique():
        ticker_group = df[df['symbol'] == symbol]
        numeric_cols = ticker_group.select_dtypes(include=['number']).columns.difference(['calendarYear', 'symbol'])
        
        for col in numeric_cols:
            not_null_data = ticker_group[ticker_group[col].notnull() & (ticker_group[col] != 0)]
            null_data = ticker_group[ticker_group[col].isnull() | (ticker_group[col] == 0)]
            
            if not not_null_data.empty and not null_data.empty:
                predictors = ['calendarYear']
                valid_data = not_null_data.dropna(subset=predictors)
                
                if len(valid_data) > 0:
                    reg = LinearRegression()
                    reg.fit(valid_data[predictors], valid_data[col])
                    
                    # Predict missing or zero values if predictors are available
                    predictors_null = null_data[predictors].dropna()
                    if not predictors_null.empty:
                        predicted_values = reg.predict(predictors_null)
                        df.loc[predictors_null.index, col] = predicted_values
    
    # Check and correct unrealistic values for gross profit margin and cost of revenue
    if 'grossProfit_to_Revenue_ratio' in df.columns:
        df.loc[df['grossProfit_to_Revenue_ratio'] > 100, 'grossProfit_to_Revenue_ratio'] = np.nan
    
    if 'costOfRevenue_to_Revenue_ratio' in df.columns:
        df.loc[df['costOfRevenue_to_Revenue_ratio'] == 0, 'costOfRevenue_to_Revenue_ratio'] = np.nan
    
    # Finalize DataFrame by sorting and potentially filling any remaining missing numeric data with 0 or other methods
    df = df.sort_values(by=['symbol', 'calendarYear'])
    
    # Optionally fill any remaining NaNs with 0 or other values
    df.fillna(0, inplace=True)
    
    return df



# Applying the function to each dataframe in the list
balancesheet_ratios_df = fill_missing_years(balancesheet_ratios_df)
cashflowstatement_ratios_df = fill_missing_years(cashflowstatement_ratios_df)
incomestatement_ratios_df = fill_missing_years(incomestatement_ratios_df)
keymetrics_ratios_df = fill_missing_years(keymetrics_ratios_df)
employeecount_ratios_df = fill_missing_years(employeecount_ratios_df)
product_revenue_df = fill_missing_years(product_revenue_df)
geo_revenue_df = fill_missing_years(geo_revenue_df)

balancesheet_ratios_df = impute_missing_data(balancesheet_ratios_df)
cashflowstatement_ratios_df = impute_missing_data(cashflowstatement_ratios_df)
incomestatement_ratios_df = impute_missing_data(incomestatement_ratios_df)
keymetrics_ratios_df = impute_missing_data(keymetrics_ratios_df)
employeecount_ratios_df = impute_missing_data(employeecount_ratios_df)
product_revenue_df = impute_missing_data(product_revenue_df)
geo_revenue_df = impute_missing_data(geo_revenue_df)



In [74]:
balancesheet_ratios_df.info()
incomestatement_ratios_df.info()
cashflowstatement_ratios_df.info()
keymetrics_ratios_df.info()
employeecount_ratios_df.info() 
geo_revenue_df.info()
product_revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 88 to 175
Data columns (total 6 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   symbol                                                220 non-null    object 
 1   calendarYear                                          220 non-null    int64  
 2   totalCurrentAssets_to_totalAssets_ratio               220 non-null    float64
 3   totalNonCurrentAssets_to_totalAssets_ratio            220 non-null    float64
 4   totalCurrentLiabilities_to_totalLiabilities_ratio     220 non-null    float64
 5   totalNonCurrentLiabilities_to_totalLiabilities_ratio  220 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 12.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 88 to 175
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype 

In [75]:
for df in dfs:
    df['calendarYear'] = df['calendarYear'].astype(int)
    df['symbol'] = df['symbol'].astype(str)

# Merging all dataframes on 'calendarYear' and 'symbol'
merged_df = dfs[0]
for df in dfs[1:]:
    if 'index' in df.columns:
        df = df.drop(columns=['index'])
    merged_df = pd.merge(left=merged_df, right=df, on=['calendarYear', 'symbol'], how='outer')

# Display the merged dataframe
print(merged_df)

     index  calendarYear symbol  acquisitionsNet_to_Revenue_ratio  \
0      0.0          2023     HP                         -0.008703   
1      1.0          2022     HP                         -0.012142   
2      2.0          2021     HP                          6.741355   
3      3.0          2020     HP                          0.848738   
4      4.0          2019     HP                         -0.577561   
..     ...           ...    ...                               ...   
160    NaN          1988   CSCO                               NaN   
161    NaN          1988   INTC                               NaN   
162    NaN          1987   INTC                               NaN   
163    NaN          1986   INTC                               NaN   
164    NaN          1985   INTC                               NaN   

     investmentsInPropertyPlantAndEquipment_to_Revenue_ratio  \
0                                           -13.767480         
1                                          

In [76]:
merged_df = impute_missing_data(merged_df)

In [77]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 88 to 175
Data columns (total 24 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   symbol                                                   220 non-null    object 
 1   calendarYear                                             220 non-null    int64  
 2   index                                                    220 non-null    float64
 3   acquisitionsNet_to_Revenue_ratio                         220 non-null    float64
 4   investmentsInPropertyPlantAndEquipment_to_Revenue_ratio  220 non-null    float64
 5   commonStockIssued_to_Revenue_ratio                       220 non-null    float64
 6   debtRepayment_to_Revenue_ratio                           220 non-null    float64
 7   revenue_to_Revenue_ratio                                 220 non-null    float64
 8   costOfRevenue_to_Revenue_rati

In [79]:
merged_df = merged_df.rename(columns={"Number of Product Segments_y": "Number of Geo Segments",
                                      "Number of Product Segments_x":"Number of Product Segments" })


In [80]:
merged_df["Number of Product Segments"]= merged_df["Number of Product Segments"].astype("float64")
merged_df["Number of Geo Segments"]= merged_df["Number of Geo Segments"].astype("float64")


In [ ]:
impute_missing_data(merged_df)

In [148]:
df.dropna(how='any', inplace=True)

In [ ]:
df.isna().sum()

In [81]:
merged_df.to_excel("trans_lit_1980.xlsx")